In [ ]:
rea

In [1]:
## Movie Critics (Meta Critic) Scrapes
## Brady Fowler
## dbf5sd
## CS Class, Masters of Data Science
## Group Members: Matt, Jordan, Andrew

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import tqdm
import csv

In [3]:
# get titles of all movies on site
def title_getter(parselist, links = []):
    for suffix in tqdm.tqdm(parselist):
        full_url = 'http://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page='+str(suffix)
        soup = read_page(full_url)
        for link in soup.findAll('a', href=True):
            href = str(link['href'].strip())
            if href[:7]=='/movie/':
                links.append(href)
    links = set(links)
    with open('/Users/bradyfowler/Documents/Summer Session/CS 5010/metacritic/movie_urls.txt', 'a') as txtfile:
        writer = csv.writer(txtfile, delimiter = '|', escapechar='\\')
        for link in links:
            writer.writerow([link])
    return(links)

In [4]:
# extract html soup from movie page
def read_page(url):
    header_info = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)'}
    # see http://www.whoishostingthis.com/tools/user-agent/ for header info
    response = requests.get(url,headers=header_info)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [5]:
# extract details from html soup
def extract_movie_deets(soup, current_url):
    try:
        critic_rating_val = soup.find(itemprop="ratingValue").get_text().strip()
    except:
        critic_rating_val ='Unknown'
    try:
        critic_rating_num = soup.find(itemprop="reviewCount").get_text().strip()
    except:
        critic_rating_num = 'Unknown'
    try:
        user_rating = soup.find('a', {'class' : 'metascore_anchor', 'href' : current_url + '/user-reviews'}).get_text().strip()
    except:
        user_rating = 'Unknown'
    try:
        user_rating_num = soup.find_all('span', {'class' : 'count'})[1].get_text().replace("- based on ","").replace(" Ratings","")
    except:
        user_rating_num = 'Unknown'
    try:
        mpaa_rating=soup.find('span', {'class' : 'data', 'itemprop' : 'contentRating'}).get_text().strip().replace('\t',' ').replace('\n',' ')
    except:
        mpaa_rating='Unknown'
    try:
        genres = []
        raw_list = soup.find_all('span', {'class' : 'data', 'itemprop' : 'genre'})
        for genre in raw_list:
            genres.append(genre.get_text().strip().replace('\t',' ').replace('\n',' '))
    except:
        genres = ['Unknown']
    try:
        movie_date = soup.find('span', {'class' : 'data', 'itemprop' : 'datePublished'}).get_text().strip().replace('\t',' ').replace('\n',' ')
    except:
        movie_date = 'Unknown'
    try:
        movie_name = soup.find('span', {'itemprop' : 'name'}).get_text().strip().replace('\t',' ').replace('\n',' ')
    except:
        movie_name = 'Unknown'
    try:
        description = soup.find('span', {'itemprop' : 'description'}).get_text().strip().replace('\t',' ').replace('\n',' ')
    except:
        description = 'Unknown'
    #return [critic_rating_val, critic_rating_num, user_rating, user_rating_num, movie_date, movie_name, description, current_url]
    #print(critic_rating_val, critic_rating_num, user_rating, user_rating_num, movie_date, movie_name, description, current_url)
    
    with open('/Users/bradyfowler/Documents/Summer Session/CS 5010/metacritic/movie_details.txt', 'a') as txtfile:
        writer = csv.writer(txtfile, delimiter = '|', escapechar='\\')
        writer.writerow([critic_rating_val, critic_rating_num, user_rating, user_rating_num, mpaa_rating, genres, movie_date, movie_name, description, current_url])

In [6]:
#example_movies = ['/movie/the-sculptress', '/movie/from-justin-to-kelly', '/movie/glitter', '/movie/dungeons-dragons', '/movie/don-peyote', '/movie/the-in-crowd', '/movie/cabin-fever-2016', '/movie/love-wedding-marriage', '/movie/daddy-day-camp', '/movie/fascination', '/movie/fair-game', '/movie/freddy-got-fingered', '/movie/a-beautiful-life', '/movie/paul-blart-mall-cop-2', '/movie/down-to-you', '/movie/new-best-friend', '/movie/avengers', '/movie/nothing-left-to-fear', '/movie/strange-wilderness', '/movie/i-will-follow-you-into-the-dark-2013', '/movie/cocktail', '/movie/left-behind-2014', '/movie/slackers', '/movie/the-adventures-of-pluto-nash', '/movie/the-master-of-disguise', '/movie/kings-ransom', '/movie/persecuted', '/movie/3-strikes', '/movie/mortal-kombat-annihilation', '/movie/date-movie', '/movie/pinocchio', '/movie/scary-movie-5', '/movie/how-to-make-love-like-an-englishman', '/movie/whipped', '/movie/born-to-be-a-star', '/movie/battlefield-earth-a-saga-of-the-year-3000', '/movie/the-tortured-1969', '/movie/superbabies-baby-geniuses-2', '/movie/alone-in-the-dark', '/movie/atlas-shrugged-iii-who-is-john-galt', '/movie/meet-the-spartans', '/movie/dirty-love', '/movie/state-property', '/movie/among-ravens', '/movie/septic-man', '/movie/transylmania', '/movie/miss-march', '/movie/screwed', '/movie/the-hottie-the-nottie', '/movie/baby-geniuses', '/movie/national-lampoons-gold-diggers', '/movie/the-human-centipede-iii-final-sequence', '/movie/vulgar', '/movie/strippers', '/movie/hillarys-america-the-secret-history-of-the-democratic-party', '/movie/the-singing-forest', '/movie/united-passions', '/movie/bio-dome', '/movie/chaos']
example_movies =['/movie/the-sculptress']
#len(example_movies)

In [7]:
# extract movies details from list of movie url
# use soup getter and soupe parser
def movie_extraction(movie_list):
    for url in tqdm.tqdm(movie_list):
        full_url = 'http://www.metacritic.com'+url
        soup = read_page(full_url)
        extract_movie_deets(soup, url)
    print('Scrape complete!')

# run on all movies

In [8]:
# get all movie urls
movie_list = title_getter(range(1,93))
len(movie_list)

100%|██████████| 92/92 [18:16<00:00, 10.48s/it]


9202

In [9]:
# read movies back into python
movie_list=[]
with open('/Users/bradyfowler/Documents/Summer Session/CS 5010/metacritic/movie_urls.txt', "r") as f:
    for movie in f:
        movie_list.append(movie.strip())  

In [10]:
import smtplib

def notify(count):
    to = 'fowler.brady@gmail.com'
    gmail_user = 'bfowl.python@gmail.com'
    gmail_pwd = 'pythonnotification'
    smtpserver = smtplib.SMTP("smtp.gmail.com",587)
    smtpserver.ehlo()
    smtpserver.starttls()
    smtpserver.ehlo
    smtpserver.login(gmail_user, gmail_pwd)
    header = 'To:' + to + '\n' + 'From: ' + gmail_user + '\n' + 'Subject: Query Done: '+str(count)+' results! \n'
    msg = header + '\n This was an automated message. \n\n'
    smtpserver.sendmail(gmail_user, to, msg)
    print('Sent!')
    smtpserver.close()

In [11]:
# extract details for all movies
movie_extraction(movie_list)
notify(len(movie_list))

100%|██████████| 9202/9202 [4:09:52<00:00,  1.53s/it]


Scrape complete!
Sent!
